In [1]:
from datetime import date, timedelta
import os
import pickle
import sys
import pandas as pd
import numpy as np
sys.path.append("..")

from abautomator.visualizer import BasicVisualizer, AbsDiffVisualizer, RelDiffVisualizer

from bokeh.io import output_notebook
from bokeh.plotting import show
# from bokeh.models import MultiChoice, CustomJS, Button
from bokeh.layouts import column, row

output_notebook()

Loading BokehJS ...

In [2]:
# %pip install -U pandas

In [3]:
# name = "rollback_analy"
name = "inspiration_analy"
analy = pickle.load(
    open(os.path.join("..", "tests", "cache", f"{name}.p"), "rb" )
)

In [4]:
df = analy.get_rel_diff_confidence_intervals()

In [5]:
from abautomator.visualizer import utils, widgets, bars

metric_order = ['n_entered_phone', 'pct_entered_phone', 'n_granted_contacts', 'pct_granted_contacts']
cond_order = ['Video01', 'Carousel01', 'Carousel02', 'Carousel03', 'Carousel04']

df = utils.order_categories(df, metric_order, cond_order)
source = utils.convert_df_to_source(df)

In [8]:
vis = RelDiffVisualizer(source)
to_show = vis.get_layout()
show(to_show)